# **26** Static Checking of Pattern Matching

There can be more to static semantics than type checking.

In [1]:
let bad_empty lst =
	match lst with
	| [] -> true

File "[1]", lines 2-3, characters 1-13:
2 | .match lst with
3 | 	| [] -> true
Warning 8: this pattern-matching is not exhaustive.
Here is an example of a case that is not matched:
_::_


val bad_empty : 'a list -> bool = <fun>


See that warning? That's because we're missing anything match with a non empty list. What happens if our pattern doesn't match?

In [2]:
bad_empty [];;

- : bool = true


In [3]:
bad_empty [1; 2; 3;];;

error: runtime_error

Uh oh! That's a bad exception!

We can fix our empty function by making our matching exhaustive:

In [4]:
let better_empty lst =
	match lst with
	| [] -> true
	| _ -> false;;

val better_empty : 'a list -> bool = <fun>


In [5]:
better_empty [1; 2; 3];;

- : bool = false


What about this code?

In [6]:
let rec bad_sum lst =
	match lst with
	| h :: t -> h + bad_sum t
	| [x] -> x
	| [] -> 0

File "[6]", line 4, characters 3-6:
4 | 	| [x] -> x
       ^^^
Warning 11: this match case is unused.


val bad_sum : int list -> int = <fun>


Our match case `[x]` is unused! But why? `[x]` can also be written as `x :: []` (pronounced "x cons nils"), so it will always match the first pattern before the second.

In [7]:
bad_sum [1; 2; 3];;

- : int = 6


In this case, as shown above, our code still works, but it's ugly :(. We should fix this:

In [8]:
let rec better_sum lst =
	match lst with
	| h :: t -> h + better_sum t
	| _ -> 0;;

val better_sum : int list -> int = <fun>


In [9]:
better_sum [1; 2; 3];;

- : int = 6


There! No error, and our code is prettier!

In [10]:
let rec another_bad_sum lst =
	List.hd lst + another_bad_sum (List.tl lst);;

val another_bad_sum : int list -> int = <fun>


In [11]:
another_bad_sum [1; 2; 3];;

error: runtime_error

What's happening here? Well, the functions `List.hd` and `List.tl` return the head and tail of a list; however, there's no head and tail of an empty list so these functions throw an exception. Here, the programmer forgets that the list might be empty. This is why it's dangerous to use `List.hd` and `List.tl`, and better as a practice to use pattern matching.

Static checking helps us write better code.

The OCaml compiler checks for, in addition to type correctness,
- Exhaustiveness of patterns
- Unused branches

This is another example of static semantics.

**Do not ignore these warnings** The compiler is finding errors for you and giving you a chance to fix them!